This notebook runs `international_empirical.jl` in a notebook environment.

In [5]:
"""
Quantify gains from changes in S and H across countries and time
	 Based on baseline Murphy and Topel model with observed H and S curves
"""

cd(ENV["INTGPATH"])

using Statistics, Parameters, DataFrames, DataVoyager
using QuadGK, NLsolve, Roots, FiniteDifferences, Interpolations
using Plots, XLSX, ProgressMeter, Formatting, TableView, Latexify, LaTeXStrings

# Import functions
include("src/TargetingAging.jl")

# Plotting backend
gr()

Plots.GRBackend()

In [6]:
"""
Import data
"""
# Import GDP data
GDP_df = CSV.read("data/GBD/real_gdp_data.csv", DataFrame, ntasks=1)
GDP_df.location_name = string.(GDP_df.location_name)
GDP_df.real_gdp_lc[(GDP_df.real_gdp_lc.=="NA")] .= "NaN"
GDP_df.real_gdp_usd[(GDP_df.real_gdp_usd.=="NA")] .= "NaN"
GDP_df.real_gdp_lc = parse.([Float64], GDP_df.real_gdp_lc)
GDP_df.real_gdp_usd = parse.([Float64], GDP_df.real_gdp_usd)
# FX rate
GDP_df.fx_rate = GDP_df.real_gdp_lc ./ GDP_df.real_gdp_usd

# Import empirical mortality curve by cause
mort_df = CSV.read("data/GBD/mortality_data.csv", DataFrame, ntasks=1)
# Import empirical disability curve by cause
health_df = CSV.read("data/GBD/health_data.csv", DataFrame, ntasks=1)
# Population structure can be staken from mortality data
pop_df = mort_df[:, [:location_name, :year, :age, :population]]

Voyager(mort_df)

Voyager(Electron.Window(Application(Base.PipeEndpoint(RawFD(21) paused, 0 bytes waiting), Process(setenv(`/Users/afajardo-ramirez/.julia/artifacts/12f3018147190ddc494f686e5fbefe8d84f16efb/Julia.app/Contents/MacOS/Julia /Users/afajardo-ramirez/.julia/packages/Electron/pOy6n/src/main.js /var/folders/l8/0dzstvdd6cl4mjpfz_6vfchc0000gp/T/jlel-65e1e40aa2a111f0049b433a775861e9 /var/folders/l8/0dzstvdd6cl4mjpfz_6vfchc0000gp/T/jlel-sn-65e21010a2a111f00d807395eef24081 XNxK9UOl7kaFIOzQOYGoLoSb4QCaZTRNyWn2Obgh2wrbDaMPV2K4sewQVcrHRh4DyKXTM3Cj3EFRuw3UHVgvDclMZGu/taMy+kdEIaOTQ7qbpzxkogByT3jjSYewyrWpS4rEzSnzsj7LCoOHfSECSRX8WmeeQKqj7eJQnbqHJo4=`,["XPC_FLAGS=0x0", "LSCOLORS=Gxfxcxdxbxegedabagacad", "PATH=/Users/afajardo-ramirez/.juliaup/bin:/Users/afajardo-ramirez/.local/bin:/Users/afajardo-ramirez/miniconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/v